## Exploring data with SQL

In this project, I'll be working with data from the CIA World Factbook. It is a huge database compiling data, statistics and information regarding all the countries around the world. I will be running some SQL queries on the data to tease out some interesting bits to share with you. 

In [2]:
import sqlite3

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [4]:
%%sql
SELECT *
    FROM facts
  LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Looking at the first few lines of our dataset, we can observe that it contains information such as land area, population size, birth/death rates and other geographical/population-related data for many different countries. Let's continue to query our database to extract more valuable information.

In [5]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
FROM facts;

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


Wanting to discover the largest and smallest populations sizes, along with the largest and smallest population growth rates, we have instead discovered something fishy with the data. Population can't be zero and neither can populaton growth be zero. There must be some kind of mistake! And also, it isn't possible for a country to have a population of over 7 billion. Now, let us look into the countries that have these erroneous data.

In [2]:
%%sql
SELECT name FROM facts
WHERE population=(SELECT MIN(population) FROM facts)

 * sqlite:///factbook.db
Done.


name
Antarctica


In [10]:
%%sql
SELECT name FROM facts
WHERE population=(SELECT MAX(population) FROM facts)

 * sqlite:///factbook.db
Done.


name
World


Well, it turns out that there wasn't a mistake. Antarctica doesn't having anyone living there, so the area having a population of zero is not at all surprising. Also, the country having the population of over 7 billion is not actually a country, but the total world population. Now that we know about these special rows/values, let us properly compute the summary statistics for the dataset. 

In [3]:
%%sql
SELECT MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
FROM facts
WHERE name!='Antarctica' AND name!='World'

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
48,1367485388,0.0,4.02


It seems like all the values look quite right now, except that for minimum population growth. This means that there is another country other than Antarctica which has 0 population growth. Let's find out what it is.

In [4]:
%%sql
SELECT name FROM facts
WHERE population_growth=0

 * sqlite:///factbook.db
Done.


name
Holy See (Vatican City)
Cocos (Keeling) Islands
Greenland
Pitcairn Islands


It seems odd that the places above have 0 population growth associated with them, because there are clearly sizeable living populations in the Vatican City as well as Greenland. But perhaps due to the population sizes being quite small, the dataset has approximated the population growth rate to be zero. Now that we have gotten the minimum population growth problem out of the way, let's display the other summary statistics together with their respective countries.

In [7]:
%%sql
SELECT name, MIN(population)
FROM facts
WHERE name!='Antarctica'

 * sqlite:///factbook.db
Done.


name,MIN(population)
Pitcairn Islands,48


In [9]:
%%sql
SELECT name, MAX(population)
FROM facts
WHERE name!='World'

 * sqlite:///factbook.db
Done.


name,MAX(population)
China,1367485388


In [10]:
%%sql
SELECT name, MAX(population_growth)
FROM facts

 * sqlite:///factbook.db
Done.


name,MAX(population_growth)
South Sudan,4.02


From the queries above, we can see that China has the world's largest population (1.3 billion) and Pitcairn Islands, a small group of islands found near Britain has the smallest population of 48. The country with the highest population growth rate is South Sudan, found in Africa. Moving on, let's calculate the average population and area of a country. 

In [11]:
%%sql
SELECT AVG(population), AVG(area)
FROM facts

 * sqlite:///factbook.db
Done.


AVG(population),AVG(area)
62094928.32231405,555093.546184739


The average country's population is around 62 million and the average land area of a country is around five hundred and fifty-five thousand square kilometers. Now that we know the average population and average area, let's come up with a few queries that explore countries with populations and areas which are above and below these values.

In [2]:
%%sql
SELECT name FROM facts
WHERE population>(SELECT AVG(population) FROM facts) AND area<(SELECT AVG(area) FROM facts)

 * sqlite:///factbook.db
Done.


name
Bangladesh
Germany
Japan
Philippines
Thailand
United Kingdom
Vietnam


Instead of coming up with a list of countries with population greater than the average, a list of countries with population lower than the average, and doing the same for area, I have decided to zoom in on a few of the more interesting countries, where population size is greater than average, but land area is smaller than average. 

These countries are jam packed with people, and it is certainly not surprising to see countries such as Bangladesh and Philippines in this list, where populations have exploded in recent times. Now let's take a look at the opposite scenario, where population size is lower than average, and area is greater than average. 

In [3]:
%%sql
SELECT name FROM facts
WHERE population<(SELECT AVG(population) FROM facts) AND area>(SELECT AVG(area) FROM facts)

 * sqlite:///factbook.db
Done.


name
Afghanistan
Algeria
Angola
Argentina
Australia
Bolivia
Botswana
Burma
Canada
Central African Republic


And we observe that there is a long list of underpopulated and very spacious countries. I didn't expect that some of these countries would be on this list, but the pretty obvious ones include Greenland, Australia, and perhaps Kazakhstan, as these places are known for vast expanses of natural scenery.

People have often speculated that the next world wars would be fought over water, as water is fast becoming a diminishing resource. Let's look at the country with the highest water area to land area ratio, followed by the country with the lowest water to land ratio. 

In [4]:
%%sql
SELECT name, MAX(area_water/area_land)
FROM facts

 * sqlite:///factbook.db
Done.


name,MAX(area_water/area_land)
British Indian Ocean Territory,905


In [5]:
%%sql
SELECT name, MIN(area_water/area_land)
FROM facts


 * sqlite:///factbook.db
Done.


name,MIN(area_water/area_land)
Afghanistan,0


And our queries tell us that the British Indian Ocean Territory, which is a small collection of British Islands located in the Indian Ocean, has the greatest abundance of water with respect to land size, and Afghanistan has virtually no water spaces to speak of, hence its water to land ratio is zero. 

Let's do a few more queries to explore information regarding birth rate and death rate. 

In [6]:
%%sql
SELECT name, death_rate
FROM facts
ORDER BY death_rate DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,death_rate
Lesotho,14.89
Ukraine,14.46
Bulgaria,14.44
Guinea-Bissau,14.33
Latvia,14.31
Chad,14.28
Lithuania,14.27
Namibia,13.91
Afghanistan,13.89
Central African Republic,13.8


It turns out that Lesotho, a country in South Africa has the highest death rate. Most of the countries on this list seem to come from Eastern Europe and Africa, seeming to reflect that the living conditions in these places are perhaps subpar.

In [7]:
%%sql
SELECT name, death_rate, birth_rate
FROM facts
WHERE death_rate > birth_rate
ORDER BY death_rate DESC;

 * sqlite:///factbook.db
Done.


name,death_rate,birth_rate
Ukraine,14.46,10.72
Bulgaria,14.44,8.92
Latvia,14.31,10.0
Lithuania,14.27,10.1
Russia,13.69,11.6
Serbia,13.66,9.08
Belarus,13.36,10.7
Hungary,12.73,9.16
Moldova,12.59,12.0
Estonia,12.4,10.51


The above query explores countries where the death rate is higher than birth rate. This means that in the long run, the countries above will face a decline in population. But perhaps it is easier to display the above information in terms of ratios.

In [8]:
%%sql
SELECT name, death_rate, birth_rate, ROUND(death_rate/birth_rate,2) AS ratio
FROM facts
WHERE death_rate > birth_rate
ORDER BY ratio DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,death_rate,birth_rate,ratio
Bulgaria,14.44,8.92,1.62
Serbia,13.66,9.08,1.5
Latvia,14.31,10.0,1.43
Lithuania,14.27,10.1,1.41
Hungary,12.73,9.16,1.39
Monaco,9.24,6.65,1.39
Germany,11.42,8.47,1.35
Slovenia,11.37,8.42,1.35
Ukraine,14.46,10.72,1.35
Saint Pierre and Miquelon,9.72,7.42,1.31


In Bulgaria, for every 1.62 deaths in the country, 1 new child is born, and for Serbia, for every 1.5 deaths in the country, 1 new child is born, so on and so forth. Now it becomes a lot more obvious to us how dire the situation in these countries is. 

Let's not end on such a grim note, and for our last query, let's explore which countries will add the most number of people to their population in the following year.

In [13]:
%%sql
SELECT name, population_growth*population AS new_addition
FROM facts
WHERE NAME!='World'
ORDER BY 2 DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,new_addition
India,1527068612.48
China,615368424.6
Nigeria,444827037.20000005
Pakistan,290665336.62
Ethiopia,287456216.91
Bangladesh,270332392.0
United States,250667713.92000002
Indonesia,235514180.08
"Congo, Democratic Republic of the",194469083.20000002
Philippines,162607385.36


And India leads the way, adding more than a billion new people in the coming year, followed by China adding more than 600 million new people in the coming year. Many other African countries are also on the above list, perhaps due to the fact that it is very difficult to regulate birth rates in these areas.